In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from datetime import datetime

In [2]:
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
conf.set('spark.sql.sources.partitionOverwriteMode', 'dynamic')

In [3]:
spark = SparkSession.builder\
    .appName('twitter-cleaning')\
    .config(conf=conf)\
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-17df70e1-c0d6-4db7-8fd8-405209df909d;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 286ms :: artifacts dl 8ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------------------

22/08/06 01:27:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = spark.read.json("s3a://projeto-pi-datalake-raw/twitter/twitter-2022-06-06.json")

22/08/06 01:28:01 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- created_at: string (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- username:

In [6]:
with open("./stopwords.txt", "r") as file:
        lines = file.readlines()
        stopwords = [l.replace("\n", "") for l in lines]

In [7]:
with open("./lexicon.txt") as file:
    lines = file.readlines()
lexicon = []
for l in lines:
    splited = l.split()
    word, polarity = splited[2], splited[5]
    word, polarity = word.split("=")[1], polarity.split("=")[1]
    lexicon.append({'word': word, 'polarity': polarity})

In [8]:
lexicon = spark.createDataFrame(data=lexicon)

In [9]:
df = (
    df
    .select(f.explode('data').alias('tweet'))
    .select('tweet.text')
    .select(f.split('text', ' ').alias('text'))
    .select(f.explode('text').alias('text'))
    .select(f.regexp_replace('text', r'[^a-zA-Z]', '').alias('text'))
    .filter(~f.col('text').isin(stopwords))   
)

In [10]:
df = (
    df
    .join(lexicon, df.text == lexicon.word, 'left')
    .fillna(value='neutral', subset=['polarity'])
    .select('text', 'polarity')
)

In [11]:
df.write.csv('s3a://projeto-pi-datalake-analytics/twitter-sentiment', header=True)

22/08/06 01:28:14 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


22/08/06 01:28:17 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
